This notebook demonstrates a *simplified* workflow for analysing and optimising a hypothetical portfolio modelled on publicly‑disclosed trades by U.S. House Speaker **Nancy Pelosi**, inspired by the @quantscience_ “1400 %” thread.

> ⚠️ *Educational use only.*  
> This example does **not** constitute financial advice and ignores factors such as taxes, commissions, leverage, and options trading.


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10,6)


In [ ]:
# ------------------------------------------------------------------
# 1️⃣  User inputs – adjust as desired
# ------------------------------------------------------------------
STOCKS      = ['AAPL', 'MSFT', 'NVDA', 'TSLA']   # Pelosi‑style tech tilt
START_DATE  = '2015-01-01'
END_DATE    = '2025-06-30'
RISK_FREE   = 0.02                               # 2 % annual T‑bill proxy
PORTFOLIOS  = 10_000                             # Monte‑Carlo samples


In [ ]:
# ------------------------------------------------------------------
# 2️⃣  Fetch adjusted close prices
# ------------------------------------------------------------------
prices = yf.download(STOCKS, start=START_DATE, end=END_DATE)['Adj Close']
prices.head()


In [ ]:
# ------------------------------------------------------------------
# 3️⃣  Compute daily log‑returns
# ------------------------------------------------------------------
returns = np.log(prices / prices.shift(1)).dropna()
annualised_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252
annualised_returns


In [ ]:
# ------------------------------------------------------------------
# 4️⃣  Helper functions
# ------------------------------------------------------------------
def portfolio_performance(weights):
    ret = np.dot(weights, annualised_returns)
    vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return ret, vol

def sharpe_ratio(weights):
    ret, vol = portfolio_performance(weights)
    return (ret - RISK_FREE) / vol


In [ ]:
# ------------------------------------------------------------------
# 5️⃣  Monte‑Carlo portfolio simulation
# ------------------------------------------------------------------
results = np.zeros((3, PORTFOLIOS))
weights_record = []

for i in range(PORTFOLIOS):
    w = np.random.random(len(STOCKS))
    w /= w.sum()
    ret, vol = portfolio_performance(w)
    results[0, i] = ret
    results[1, i] = vol
    results[2, i] = (ret - RISK_FREE) / vol
    weights_record.append(w)

max_sharpe_idx = results[2].argmax()
opt_weights   = weights_record[max_sharpe_idx]
opt_ret, opt_vol = portfolio_performance(opt_weights)
opt_sharpe = (opt_ret - RISK_FREE) / opt_vol

print("Optimal Weights:")
for s, w in zip(STOCKS, opt_weights):
    print(f"  {s:<5}: {w:.2%}")

print(f"Expected Annual Return   : {opt_ret:.2%}")
print(f"Expected Annual Volatility: {opt_vol:.2%}")
print(f"Sharpe Ratio             : {opt_sharpe:.2f}")


In [ ]:
# ------------------------------------------------------------------
# 6️⃣  Plot Efficient Frontier
# ------------------------------------------------------------------
plt.scatter(results[1, :], results[0, :], c=results[2, :], cmap='viridis', alpha=0.6)
plt.colorbar(label='Sharpe Ratio')
plt.scatter(opt_vol, opt_ret, marker='*', s=250, c='red', label='Optimal')
plt.xlabel('Volatility (σ)')
plt.ylabel('Return (μ)')
plt.title('Efficient Frontier')
plt.legend(); plt.grid(True)


In [ ]:
# ------------------------------------------------------------------
# 7️⃣  Simulate growth of $100 k using optimal portfolio
# ------------------------------------------------------------------
initial_investment = 100_000
years = (pd.to_datetime(END_DATE) - pd.to_datetime(START_DATE)).days / 365.25
final_value = initial_investment * (1 + opt_ret) ** years
print(f"Hypothetical Final Value: ${final_value:,.2f} over {years:.1f} years")
print(f"Total Return            : {(final_value/initial_investment - 1):.2%}")


> ### ⚖️ Caveats & Ethics  
> - Congressional trades are public via the STOCK Act disclosures but **delay** can be months.  
> - Outperformance may stem from privileged information or skilful risk‑taking; conclusions are speculative.  
> - Real portfolios incur slippage, taxes, commissions, and may employ options/leverage not modelled here.  
